# Notebook for data acquisition functions


#### The first fctn take he target year and a filepath as argument to download the data from a specific season and put it in a certain path

In [24]:
import pandas as pd
import requests
import os
from tqdm.notebook import tqdm

def get_season_data(year, filepath):
    YEAR = year
    DIRECTORY  = f"{filepath}/JSON/{YEAR}/"
    os.makedirs(DIRECTORY, exist_ok=True)
    seasons = ["01", "02", "03", "04"]
    for s in seasons:
        for g in tqdm(range(1,1300)):
            game_number = str(g).zfill(4)
            GAME_ID = f"{YEAR}{s}{game_number}"
            PATH = f"{DIRECTORY}/{GAME_ID}.json"
            if not os.path.isfile(PATH):            
                r = requests.get(f"https://statsapi.web.nhl.com/api/v1/game/{GAME_ID}/feed/live/")
                if r.status_code == 200:
                    with open(PATH, 'wb') as f:
                        f.write(r.content)  
            else:
                print("File already exists")

In [ ]:
get_season_data(2018, "../ift6758/data/")

  0%|          | 0/1299 [00:00<?, ?it/s]

File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists


  0%|          | 0/1299 [00:00<?, ?it/s]

File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists


  0%|          | 0/1299 [00:00<?, ?it/s]

File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists


  0%|          | 0/1299 [00:00<?, ?it/s]

## From the api documentation the max of matches per season is less than 1300. For security, we put 1300 to check all the ids where we can find a match on the data. We could have put a break after we have a 404 answer from the request (for efficency). But we assumed that the data might miss some game ids. And we download them just once. So no need for time efficiency for now

404